Weather affects every single human on earth for the better or worse, and we've come to rely on weather predictions in order to plan how we spend our day. But how can we predict the weather? In this post we're going to develop a machine learning model with recurrent neural networks to see how well we can predict the weather.

As per previous posts we're going to go through the following steps (typical of any machine learning project):
1. Data exploration & analysis
2. Build a model
3. Train the model
4. Evaluate the model


In [31]:
import pandas as pd
import plotly
import plotly.express as px
from IPython.core.display import HTML
import torch
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Data Exploration & Analysis

We'll be using rainfall records for Newcastle NSW retrieved from the Australian Bureau of Meteorology, this can be downloaded at: http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_nccObsCode=136&p_display_type=dailyDataFile&p_startYear=&p_c=&p_stn_num=061055

In [32]:
rainfall = pd.read_csv('data/IDCJAC0009_061055_1800_Data.csv')
print(rainfall.head(1))

  Product code  Bureau of Meteorology station number  Year  Month  Day  \
0   IDCJAC0009                                 61055  1862      1    1   

   Rainfall amount (millimetres)  \
0                            0.0   

   Period over which rainfall was measured (days) Quality  
0                                             NaN       Y  


Let's clean it up

In [33]:
rainfall['timestamp'] = pd.to_datetime(rainfall[['Year', 'Month', 'Day']])
rainfall = rainfall.drop(['Product code','Bureau of Meteorology station number','Year','Month','Day','Period over which rainfall was measured (days)'],axis=1)
rainfall = rainfall.rename(columns={"Rainfall amount (millimetres)": "rainfall", "Quality": "quality"})
rainfall.head()

,rainfall,quality,timestamp
0,0.0,Y,1862-01-01
1,0.0,Y,1862-01-02
2,0.0,Y,1862-01-03
3,0.0,Y,1862-01-04
4,0.0,Y,1862-01-05


In [34]:
print(f"First date: {rainfall.timestamp.min()}, last date: {rainfall.timestamp.max()}")

First date: 1862-01-01 00:00:00, last date: 2022-04-07 00:00:00


In [35]:
rainfall = rainfall.dropna(subset=["rainfall"])
rainfall = rainfall.set_index('timestamp')

In [36]:
def generate_time_lags(df, value, n_lags):
    df_n = df.copy()
    for n in range(1, n_lags + 1):
        df_n[f"lag{n}"] = df_n[value].shift(n)
    df_n = df_n.iloc[n_lags:]
    return df_n

In [37]:
df_gen = generate_time_lags(rainfall,'rainfall', 100)

df_gen

/var/folders/7t/v3b3p3455bq_r5tknrnqgczr0000gn/T/ipykernel_9056/4251947411.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_n[f"lag{n}"] = df_n[value].shift(n)
/var/folders/7t/v3b3p3455bq_r5tknrnqgczr0000gn/T/ipykernel_9056/4251947411.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_n[f"lag{n}"] = df_n[value].shift(n)


,rainfall,quality,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,...,lag91,lag92,lag93,lag94,lag95,lag96,lag97,lag98,lag99,lag100
timestamp,,,,,,,,,,,,,,,,,,,,,
1862-04-11,1.0,Y,0.8,10.4,1.0,0.8,0.0,7.6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1862-04-12,0.0,Y,1.0,0.8,10.4,1.0,0.8,0.0,7.6,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1862-04-13,12.2,Y,0.0,1.0,0.8,10.4,1.0,0.8,0.0,7.6,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1862-04-14,1.3,Y,12.2,0.0,1.0,0.8,10.4,1.0,0.8,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1862-04-15,0.0,Y,1.3,12.2,0.0,1.0,0.8,10.4,1.0,0.8,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-03,0.0,N,0.2,16.8,3.2,6.6,2.0,5.2,0.8,6.4,...,0.0,0.0,0.0,0.0,1.2,2.4,0.0,0.0,0.0,0.2
2022-04-04,0.0,N,0.0,0.2,16.8,3.2,6.6,2.0,5.2,0.8,...,0.0,0.0,0.0,0.0,0.0,1.2,2.4,0.0,0.0,0.0
2022-04-05,0.0,N,0.0,0.0,0.2,16.8,3.2,6.6,2.0,5.2,...,0.0,0.0,0.0,0.0,0.0,0.0,1.2,2.4,0.0,0.0


In [38]:
# # https://towardsdatascience.com/building-rnn-lstm-and-gru-for-time-series-using-pytorch-a46e5b094e7b

df_features = (
                df_gen
                .assign(hour = df_gen.index.hour)
                .assign(day = df_gen.index.day)
                .assign(month = df_gen.index.month)
                .assign(day_of_week = df_gen.index.dayofweek)
                .assign(week_of_year = df_gen.index.week)
              )

/var/folders/7t/v3b3p3455bq_r5tknrnqgczr0000gn/T/ipykernel_9056/3761641247.py:9: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  .assign(week_of_year = df_gen.index.week)


In [39]:
df_features['quality'] = df_features['quality'].replace('Y',1)
df_features['quality'] = df_features['quality'].replace('N',0)

In [40]:
input_features = torch.from_numpy(df_features.loc[:, df_features.columns != 'rainfall'].to_numpy()).type(torch.float32)

output_feature = torch.from_numpy(df_features['rainfall'].to_numpy()).type(torch.float32)

data = torch.utils.data.TensorDataset(input_features, output_feature)

In [41]:
split = 0.1
rows = df_features.shape[0]
test_split = int(rows*split)
val_split = int(rows*split*2)
train_split = rows - val_split - test_split

train_set, val_set, test_set = torch.utils.data.random_split(data, [train_split, val_split, test_split])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_set) 
test_loader = torch.utils.data.DataLoader(test_set)

In [42]:
class RNNModel(torch.nn.Module):
    def __init__(self, input_dimension, hidden_dimension, layer_dimension, output_dimension, dropout_probability):
        super(RNNModel, self).__init__()

        self.hidden_dimension = hidden_dimension
        self.layer_dimension = layer_dimension

        self.rnn = torch.nn.RNN(
            input_dimension, hidden_dimension, layer_dimension, batch_first=True, dropout=dropout_probability
        )
        self.fc = torch.nn.Linear(hidden_dimension, output_dimension)

    def forward(self, x):
        # Hidden state
        h0 = torch.zeros(self.layer_dimension, x.size(0), self.hidden_dimension).requires_grad_()

        out, h0 = self.rnn(x, h0.detach())

        out = out[:, -1, :]

        out = self.fc(out)
        return out

In [43]:
class LSTMModel(torch.nn.Module):
    def __init__(self, input_dimension, hidden_dimension, layer_dimension, output_dimension, dropout_probability):
        super(LSTMModel, self).__init__()

        self.hidden_dimension = hidden_dimension
        self.layer_dimension = layer_dimension

        self.lstm = torch.nn.LSTM(
            input_dimension, hidden_dimension, layer_dimension, batch_first=True, dropout=dropout_probability
        )
        self.fc = torch.nn.Linear(hidden_dimension, output_dimension)

    def forward(self, x):
        # Hidden state
        h0 = torch.zeros(self.layer_dimension, x.size(0), self.hidden_dimension).requires_grad_()

        # LSTM Cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        out, h0 = self.rnn(x, h0.detach())

        out = out[:, -1, :]

        out = self.fc(out)
        return out

In [44]:
class GRUModel(torch.nn.Module):
    def __init__(self, input_dimension, hidden_dimension, layer_dimension, output_dimension, dropout_probability):
        super(GRUModel, self).__init__()

        self.hidden_dimension = hidden_dimension
        self.layer_dimension = layer_dimension

        self.gru = torch.nn.GRU(
            input_dimension, hidden_dimension, layer_dimension, batch_first=True, dropout=dropout_probability
        )
        self.fc = torch.nn.Linear(hidden_dimension, output_dimension)

    def forward(self, x):
        # Hidden state
        h0 = torch.zeros(self.layer_dimension, x.size(0), self.hidden_dimension).requires_grad_()

        out, h0 = self.gru(x, h0.detach())

        out = out[:, -1, :]

        out = self.fc(out)
        return out

In [45]:
class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []

    def train_step(self, x, y):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()

    def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1):
        model_path = f'models/{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([batch_size, -1, n_features])
                y_batch = y_batch
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features])
                    y_val = y_val
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch % 50 == 0):
                print(
                    f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
                )

        torch.save(self.model.state_dict(), model_path)

    def evaluate(self, test_loader, batch_size=1, n_features=1):
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features])
                y_test = y_test
                self.model.eval()
                yhat = self.model(x_test)
                predictions.append(yhat.detach().numpy())
                values.append(y_test.detach().numpy())

        return predictions, values

    def plot_losses(self):
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

In [48]:
import torch.optim as optim

input_dimension = len(train_loader)
output_dimension = 1
hidden_dimension = 64
layer_dimension = 3
batch_size = 106
dropout = 0.2
n_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {'input_dimension': input_dimension,
                'hidden_dimension' : hidden_dimension,
                'layer_dimension' : layer_dimension,
                'output_dimension' : output_dimension,
                'dropout_probability' : dropout}

model = RNNModel(**model_params)

loss_fn = torch.nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dimension)
opt.plot_losses()

predictions, values = opt.evaluate(test_loader, batch_size=1, n_features=input_dimension)

RuntimeError: shape '[106, -1, 40039]' is invalid for input of size 106